# **NYC Taxi Demand Anomaly Detection Using LSTM**
Developed an LSTM-based classifier to detect anomalies in NYC taxi demand data. The project involved time series preprocessing, sequence modeling, and training a neural network to identify unusual demand patterns. This helped build a practical understanding of deep learning techniques for anomaly detection in real-world time series data.

# Packages

In [ ]:
using Pkg
Pkg.add(["Flux", "ConformalPrediction", "DataFrames", "CSV", "Plots", "Statistics", "StatsPlots", "Random", "Measures", "RollingFunctions", "ShiftedArrays"])

In [ ]:
using Flux, ConformalPrediction, DataFrames, CSV, Plots, Statistics, StatsPlots, Random, Dates, Measures, RollingFunctions, ShiftedArrays

# Data Preprocessing

In [ ]:
data = CSV.read("/content/nyc_taxi.csv", DataFrame)

In [ ]:
data[!, "timestamp"] = DateTime.(data[!, "timestamp"], dateformat"yyyy-mm-dd HH:MM:SS.ssssss")

In [ ]:
anomalies_timestamp = [
    "2014-11-01 19:00:00.000000",
    "2014-11-27 15:30:00.000000",
    "2014-12-25 15:00:00.000000",
    "2015-01-01 01:00:00.000000",
    "2015-01-27 00:00:00.000000"
]
anomaly_times = DateTime.(anomalies_timestamp, dateformat"yyyy-mm-dd HH:MM:SS.ssssss")

**Create a column for anomalies**

In [ ]:
data[!, "is_anomaly"] .= 0
for each in anomaly_times
    row_indices = findall(data[!, "timestamp"] .== each)
    data[row_indices, "is_anomaly"] .= 1
end

# Data Description

In [ ]:
describe(data)


In [ ]:
println("Start time: ", minimum(data.timestamp))
println("End time: ", maximum(data.timestamp))
println("Time difference: ", maximum(data.timestamp) - minimum(data.timestamp))

time_diff = maximum(data.timestamp) - minimum(data.timestamp)
days_diff = time_diff.value / (1000 * 60 * 60 * 24)  # from ms to days
println("Time difference: ", round(days_diff, digits=2), " days")



**Adding New Columns**

In [ ]:
data[!, :Weekday] = dayname.(data.timestamp)
data[!, :Hour] = hour.(data.timestamp)
data[!, :Day] = dayofweek.(data.timestamp)
data[!, :Month] = month.(data.timestamp)
data[!, :Year] = year.(data.timestamp)
data[!, :Month_day] = day.(data.timestamp)

# Data Visualizing

**NYC Taxi Passenger Count based on hourly, daily and weekly**

In [ ]:
# Resample timestamps
data.hourly = DateTime.(floor.(data.timestamp, Dates.Hour))
data.daily = DateTime.(floor.(data.timestamp, Dates.Day))
data.weekly = DateTime.(floor.(data.timestamp, Dates.Week))

# Aggregate means
df_hour = combine(groupby(data, :hourly), :value => mean => :mean_value)
df_day = combine(groupby(data, :daily), :value => mean => :mean_value)
df_week = combine(groupby(data, :weekly), :value => mean => :mean_value)

# Plot styling
default(
    yformatter = :plain,
    legend = false,
    grid = true,
    gridcolor = :gray,
    gridalpha = 0.5,
    tickfontsize = 8,
    guidefontsize = 10,
    titlefontsize = 11,
    linewidth = 1.5,
    left_margin = 10mm
)

# Create subplots and assign it to a variable
p = plot(
    plot(df_hour.hourly, df_hour.mean_value, title="NYC Taxi Passengers (Hourly)", xlabel="", ylabel="Passenger Count"),
    plot(df_day.daily, df_day.mean_value, title="NYC Taxi Passengers (Daily)", xlabel="", ylabel="Passenger Count"),
    plot(df_week.weekly, df_week.mean_value, title="NYC Taxi Passengers (Weekly)", xlabel="", ylabel="Passenger Count"),
    layout = (3, 1),
    size = (700, 1200)
)

# Save the plot
#savefig(p, "subplot.png")

**Density Plot for Value Distribution**

In [ ]:
# Create a shaded density plot for 'value'
density_plot = density(data.value,
                       xformatter = :plain,
                       title="Overall Value Distribution",
                       xlabel="Value",
                       ylabel="Density",
                       linewidth=2,
                       fillrange=0.0,
                       fillalpha=0.3)


plot(density_plot, size=(700, 300), grid=true)
# Save the plot
#savefig("plot.png")

In [ ]:
grouped = combine(groupby(data, :Weekday), :value => mean => :mean_value)

# Plot the bar chart with increased space between the bars
bar(grouped.Weekday, grouped.mean_value,
    title="New York City Taxi Demand by Day",
    xlabel="Day of the Week",
    ylabel="Demand",
    legend=false,
    grid=true,
    width=0.4)  # Reduce the width to increase space between bars
# Save the plot using the assigned variable 'p'
#savefig("plot2.png")

**New York Taxi Demand by Hour**

In [ ]:
grouped_hourly = combine(groupby(data, :Hour), :value => mean => :mean_value)

# Plot the bar chart for hourly demand
bar(grouped_hourly.Hour, grouped_hourly.mean_value,
    title="New York City Taxi Demand by Hour",
    xlabel="Hour of the Day",
    ylabel="Demand",
    legend=false,
    grid=true,
    bar_width=0.3,
    size=(700, 300))
#savefig("plot1.png")

In [ ]:
grouped_hourly = combine(groupby(data, :Hour), :value => mean => :mean_value)

# Plot the line chart for hourly demand
plot(grouped_hourly.Hour, grouped_hourly.mean_value,
     title="New York City Taxi Demand by Hour",
     xlabel="Hour of the Day",
     ylabel="Demand",
     label="Hourly Demand",
     linewidth=2,
     grid=true,
     size=(700, 300))
#savefig("plot3.png")

**New York City Taxi Demand by Hour and Day**

In [ ]:
grouped = combine(groupby(data, [:Hour, :Weekday]), :value => mean => :mean_value)

# Plot the line chart with each day of the week as a separate line
p = plot(grouped.Hour, grouped.mean_value,
    group=grouped.Weekday,
    title="New York City Taxi Demand by Hour and Day",
    xlabel="Hour of the Day",
    ylabel="Average Demand",
    linewidth=2,
    grid=true,
    legend=:topright,
    size=(700, 200))

p = plot!(p, label=grouped.Weekday)

display(p)

**Anomalies Visualisation**

In [ ]:
date_ticks = data.timestamp[1:floor(Int, length(data.timestamp)/8):end]
date_ticks = date_ticks[1:8]

date_labels = Dates.format.(date_ticks, "mmm yyyy")

plot(data.timestamp, data.value,
    label="Taxi Demand",
    linewidth=1.5,
    color=:blue,
    xticks=(date_ticks, date_labels),
    xrotation=45,
    xlabel="Date",
    ylabel="Passenger Count",
    title="NYC Taxi Demand",
    legend=:topleft,
    size=(800,400))

scatter!(data.timestamp[data.is_anomaly .== 1], data.value[data.is_anomaly .== 1],
    label="Anomalies",
    color=:red,
    markersize=5)

plot!(grid=true, gridalpha=0.3)
#savefig("plot5.png")

# LSTM Model

**Normalised the dataset**

In [ ]:
X = data[!, :value]
X_normalized = (X .- mean(X)) ./ std(X)
y = data.is_anomaly

**Split Dataset**

In [ ]:
n = length(X_normalized)
n_train = floor(Int, 0.7n)

X_train = X_normalized[1:n_train]
y_train = y[1:n_train]

X_test = X_normalized[n_train+1:end]
y_test = y[n_train+1:end]


**Create Sequences**

In [ ]:
window_size = 24
function create_sequences(data, window_size)
    sequences = []
    for i in 1:length(data)-window_size
        push!(sequences, data[i:i+window_size-1])
    end
    return sequences
end

In [ ]:
X_train_seq = create_sequences(X_train, window_size)
X_test_seq = create_sequences(X_test, window_size)

X_train_flux = [reshape(x, 1, window_size, 1) for x in X_train_seq]
X_test_flux = [reshape(x, 1, window_size, 1) for x in X_test_seq]

y_train_seq = y_train[window_size+1:end]
y_test_seq = y_test[window_size+1:end]


In [ ]:
y_train_seq = y_train[window_size+1:end]
y_test_seq = y_test[window_size+1:end]

**LSTM Model**

In [ ]:
model = Chain(
    LSTM(1 => 32),
    Dropout(0.1),
    LSTM(32 => 16),
    Dense(16 => 1, sigmoid)
)


Chain(
  LSTM(1 => 32),                        # 4_352 parameters
  Dropout(0.1),
  LSTM(32 => 16),                       # 3_136 parameters
  Dense(16 => 1, σ),                    # 17 parameters
)                   # Total: 8 arrays, 7_505 parameters, 29.895 KiB.

**Weighted Loss Function**

In [ ]:
function weighted_loss(y_pred, y_true; weight_anomaly=200.0f0)
    ϵ = eps(Float32)
    -mean(weight_anomaly * y_true .* log.(y_pred .+ ϵ) .+
          (1 .- y_true) .* log.(1 .- y_pred .+ ϵ))
end


function loss(m, x, y)
    y_pred = m(x)
    weighted_loss(y_pred, Float32(y))
end

In [ ]:
X_train_float = [Float32.(x) for x in X_train_flux]
y_train_float = Float32.(y_train_seq)

**Optimiser**

In [ ]:
opt_state = Flux.setup(Adam(0.001), model)

**Training**

In [ ]:
using Flux.Optimise: train!
epochs = 10
for epoch in 1:epochs
    Flux.train!(loss, model, zip(X_train_float, y_train_float), opt_state)

    train_pred = [Flux.flatten(model(x))[1] for x in X_train_float]
    current_loss = mean([loss(model, x, y) for (x, y) in zip(X_train_float, y_train_float)])
    current_acc = mean(round.(train_pred) .== y_train_float) * 100

    println("Epoch $epoch: Loss = ", round(current_loss, digits=4),
            " | Accuracy = ", round(current_acc, digits=4), "%")
end

Epoch 1: Loss = 0.6552 | Accuracy = 99.9722%
Epoch 2: Loss = 0.4111 | Accuracy = 99.9722%
Epoch 3: Loss = 0.2561 | Accuracy = 99.9722%
Epoch 4: Loss = 0.2346 | Accuracy = 99.9722%
Epoch 5: Loss = 0.2184 | Accuracy = 99.9722%
Epoch 6: Loss = 0.2117 | Accuracy = 99.9722%
Epoch 7: Loss = 0.2147 | Accuracy = 99.9722%
Epoch 8: Loss = 0.2067 | Accuracy = 99.9722%
Epoch 9: Loss = 0.2202 | Accuracy = 99.9722%
Epoch 10: Loss = 0.192 | Accuracy = 99.9722%


**Model Evaluation**

In [ ]:
function evaluate(model, X_test, y_test)
    predictions = [model(x) for x in X_test]

    predictions = [Flux.flatten(pred)[1] for pred in predictions]
    predictions = [pred > 0.5 ? 1 : 0 for pred in predictions]

    accuracy = sum(predictions .== y_test) / length(y_test)
    return accuracy
end

# Evaluate on test data
accuracy = evaluate(model, X_test_flux, y_test_seq)
println("Test Accuracy: ", accuracy)

In [ ]:
X_train_float = [Float32.(x) for x in X_train_flux]
y_train_float = Float32.(y_train_seq)

X_test_float = [Float32.(x) for x in X_test_flux]

**Confusion Matrix**

In [ ]:
 threshold = 0.03
test_preds = [Flux.flatten(model(x))[1] for x in X_test_float]
test_binary = map(p -> p > threshold ? 1 : 0, test_preds)

TP = sum((y_test_seq .== 1) .& (test_binary .== 1))
FP = sum((y_test_seq .== 0) .& (test_binary .== 1))
FN = sum((y_test_seq .== 1) .& (test_binary .== 0))
TN = sum((y_test_seq .== 0) .& (test_binary .== 0))

println("True Positive: $TP,
 False Positive: $FP,
 False Negative: $FN,
 True Negative: $TN")

In [ ]:
TP = 2
FP = 1284
FN = 1
TN = 1786

# Calculate precision, accuracy, recall and f1-score
precision = TP / (TP + FP)
accuracy = (TP + TN) / (TP + TN + FP + FN)
recall = TP / (TP + FN) # Calculate recall
my_f1score = 2 * (precision * recall) / (precision + recall)

# Print the calculated metrics
println("Precision: $precision")
println("Accuracy: $accuracy")
println("Recall: $recall") # Print recall
println("F1-score: $my_f1score") # Print F1-score

# Plotting
labels = ["TP", "FP", "FN", "TN"]
values = [TP, FP, FN, TN]

# Create the confusion matrix plot
p1 = bar(labels, values, color=["green", "red", "orange", "blue"],
         title="Confusion Matrix", ylabel="Count", xlabel="Categories")

# Plot Precision and Accuracy
p2 = bar(["Precision", "Accuracy", "Recall", "F1-score"], [precision, accuracy, recall, my_f1score], color=["blue", "green", "orange", "red"],
         title="Metrics", ylabel="Score")

# Show the plots
plot(p1, p2, layout=(1, 2))
#savefig("plot4.png")